# Import data

Import the datset, replacing the file path with your own. 

In [20]:
import pandas as pd
MyDeal = pd.read_excel(r"C:\Users\justi\OneDrive\Documents\UT_Capstone_Data_Set (2).xlsx") 

In [63]:
import warnings
warnings.filterwarnings('ignore')

# Specify won and lost deals as 0,1

In [64]:

import numpy as np

Train = MyDeal.loc[(MyDeal.DealStatusValue == 'Won') | (MyDeal.DealStatusValue == 'Lost')]

# Test = MyDeal.loc[(MyDeal.DealStatusValue == 'Deliverable Provided')| (MyDeal.DealStatusValue == 'Solutioning')]


Train['Won'] = np.where(Train['DealStatusValue']== 'Lost', 0, 1)

# Governance Status

This turns governance into a binary - either governance (1) or no governance (0). 

In [65]:
Train['Governance'] = np.where(Train['GovernanceStatusName']== 'No Governance', 0, 1)

# Collapsing deals with multiple rows

This is how we manage deals with multiple rows. This chunk combines multiple row deals, and creates a 'DealTechnologies' column simply indicating the number of technologies in a given deal. 

In [66]:
# calculate technology complexity
complexity = Train.groupby('MyDealId',as_index=True).size()
complexity_dict = complexity.to_dict()

 

## Checking for duplicate id's
mult_id_list = Train[Train.duplicated(['MyDealId'])]
mult_id_list = mult_id_list[mult_id_list['IsPrimary'] == 1]
mult_id_list = mult_id_list['MyDealId'].tolist()
Train['ComplexityFlag'] = np.where(Train['MyDealId'].isin(mult_id_list), 1, 0)

 

##Keep only primary entries
Train = Train[Train['IsPrimary'] == 1]

 

# add technolidy complexity column
Train['DealTechnologies'] = Train['MyDealId'].map(complexity_dict)

# Time to solution

Time to solution is calculated by taking the difference betweeen FinalDeliverableProvidedDate (or FirstDeliverableProvidedDate if this field is null) and IntakeDate. 

In [67]:

Train['DeliverableProvidedNew'] =  Train['FirstDeliverableProvidedDate'].mask(pd.isnull, Train['FinalDeliverableProvidedDate'])


Train['DeliverableProvidedNew'] = pd.to_datetime(Train['DeliverableProvidedNew'], errors='coerce')
Train['IntakeDates1'] = pd.to_datetime(Train['IntakeDate'], errors='coerce')


Train['TimeToSolution'] = (Train['DeliverableProvidedNew']-Train['IntakeDates1']).dt.days

def negative_replace(x):
    if x<0:
        return 0 
    else:
        return x
    
Train['TimeToSolution'] = Train['TimeToSolution'].map(negative_replace)

# Year

Seperating the year out from the intake date. 

In [68]:
#Creating Year Column
import numpy as np

Train['IntakeDates_Year'] = Train['IntakeDate'].astype(str).str[0:4]



Train['IntakeDates_Year']


1        2020
3        2020
6        2020
7        2020
9        2020
         ... 
43328    2020
43329    2020
43333    2020
43334    2020
43336    2020
Name: IntakeDates_Year, Length: 24577, dtype: object

# Quarter

Inferring fiscal quarter from intake date.

In [69]:
## Creating Quarter Variable

def month(x):
    x = x.month
    return x

Train['Fiscal_Quarter'] = Train.IntakeDate.map(month)

Train['Fiscal_Quarter'] = Train.Fiscal_Quarter.replace([2,3,4,5,6,7,8,9,10,11,12,1],[1,1,1,2,2,2,3,3,3,4,4,4])

Train['Fiscal_Quarter'] 

1        1
3        1
6        3
7        1
9        2
        ..
43328    2
43329    1
43333    1
43334    4
43336    4
Name: Fiscal_Quarter, Length: 24577, dtype: int64

# Train/Test Specification

Make sure to run this cell - it indicates that we are using all of the data as the training set. 

In [70]:
Train_data = Train

# Imputation for price vars

The cell below saves mean and median info for later transformations of test data. 

In [71]:
mean_cc = Train['ContractCost'].mean()
mean_cm = Train['ContractMargin'].mean()
med_fr = Train['ForecastedRevenue'].median()

Here we are imputuing null values for Contract Cost (mean), Contract Margin (Mean) and Forecasted Revenue (Median). 

In [72]:
#Imputation
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

Train[['ContractMargin']]=imputer.fit_transform(Train[['ContractMargin']])
Train[['ContractCost']]=imputer.fit_transform(Train[['ContractCost']])

imputer = SimpleImputer(missing_values=np.nan, strategy='median')
Train[['ForecastedRevenue']]=imputer.fit_transform(Train[['ForecastedRevenue']])


# Contract Cost Bins

This bins contract costs into 4 equally sized based on quartile. Bin 1 is low, Bin 4 is high. 

In [73]:
##Creating bins for Contract Cost


Train_data['ContractCost'] = pd.to_numeric(Train_data['ContractCost'])
Train_data['ContractCost'] = Train_data['ContractCost'].abs()


bin1 = Train_data['ContractCost'].quantile([.25]).values[0]
bin2 = Train_data['ContractCost'].quantile([.5]).values[0]
bin3 = Train_data['ContractCost'].quantile([.75]).values[0]

bins = [-.1, bin1, bin2 , bin3, 1000000000000000000000]
labels = [1,2,3,4]
Train['ContractCosts_Binned'] = pd.cut(Train['ContractCost'], bins=bins, labels=labels)


Train['ContractCosts_Binned']


1        4
3        1
6        3
7        3
9        3
        ..
43328    3
43329    3
43333    3
43334    1
43336    1
Name: ContractCosts_Binned, Length: 24577, dtype: category
Categories (4, int64): [1 < 2 < 3 < 4]

# Contract Margin Bins

This bins contract margin into 4 equally sized based on quartile. Bin 1 is low, Bin 4 is high. This is commented out because in our final model we use contract margin as a continuous variable. 

In [241]:
# #Contract Margin Bins

# Train_data['ContractMargin'] = pd.to_numeric(Train['ContractMargin'])
# # Train['ContractMargin'] = Train['ContractMargin'].abs()

# bin1 = Train_data['ContractMargin'].quantile([.25]).values[0]
# bin2 = Train_data['ContractMargin'].quantile([.5]).values[0]
# bin3 = Train_data['ContractMargin'].quantile([.75]).values[0]

# bins = [-1,0, bin1, bin2 , bin3, 1000000000000000000000]
# labels = [1,2,3,4,5]
# Train['ContractMargin_Binned'] = pd.cut(Train['ContractMargin'], bins=bins, labels=labels)


# Train['ContractMargin_Binned']


# Forecasted Revenue Bins

This bins forecasted revenue into 4 equally sized based on quartile. Bin 1 is low, Bin 4 is high. This is commented out because in our final model we omitted forecasted revenue due to overwhelming nulls. 

In [771]:
##Binning Forecasted Revenue

# Train_data['ForecastedRevenue'] = pd.to_numeric(Train['ForecastedRevenue'])
# bin1 = Train_data['ForecastedRevenue'].quantile([.04]).values[0]
# bin2 = Train_data['ForecastedRevenue'].quantile([.5]).values[0]
# bin3 = Train_data['ForecastedRevenue'].quantile([.90]).values[0]

# bins = [-.1, bin1, bin2 , bin3, 1000000000000000000000]
# labels = [1,2,3,4]
# Train['ForecastedRevenue_Binned'] = pd.cut(Train['ForecastedRevenue'], bins=bins, labels=labels)


# Train['ForecastedRevenue_Binned']

# Imputation for SourceSystem and SOW Revisions

This chunk imputes SourceSystem with the mode value (SFDC - Dell Main) and fills null SOW Revisions with 0. 

In [74]:
#Imputation

Train['SourceSystem'] = Train['SourceSystem'].fillna(Train['SourceSystem'].mode().iloc[0])
Train['NumberOfSowRevisions'] = Train.NumberOfSowRevisions.fillna(0)


In [75]:
Train['SourceSystem']

1        SFDC - Dell Main
3        SFDC - Dell Main
6        SFDC - Dell Main
7        SFDC - Dell Main
9        SFDC - Dell Main
               ...       
43328    SFDC - Dell Main
43329    SFDC - Dell Main
43333     SFDC - EMC Core
43334    SFDC - Dell Main
43336    SFDC - Dell Main
Name: SourceSystem, Length: 24577, dtype: object

# Technology Time



Here we bin technologies into 5 hierarchical bins based on average time to solution for the technology. Bin 1 is low time to solution, Bin 5 is high. The values bin1, bin2, bin3, bin4 will show the boundaries of the bins. 

In [76]:
df = Train_data.groupby(['TechnologyName']).mean('TimeToSolution')
bin1 = df['TimeToSolution'].quantile([.20])
bin2 = df['TimeToSolution'].quantile([.40])
bin3 = df['TimeToSolution'].quantile([.60])
bin4 = df['TimeToSolution'].quantile([.80])

bin_one = df[(df['TimeToSolution'] > -0.1) & (df['TimeToSolution'] <= bin1.iloc[0])].index
bin_two = df[(df['TimeToSolution'] > bin1.iloc[0]) & (df['TimeToSolution'] <= bin2.iloc[0])].index
bin_three = df[(df['TimeToSolution'] > bin2.iloc[0]) & (df['TimeToSolution'] <= bin3.iloc[0])].index
bin_four = df[(df['TimeToSolution'] > bin3.iloc[0]) & (df['TimeToSolution'] <= bin4.iloc[0])].index
bin_five = df[(df['TimeToSolution'] > bin4.iloc[0]) & (df['TimeToSolution'] <= 10000000000)].index
Train['TechnologyComplexity'] = 0

for i in range (0,len(Train)):
    if Train['TechnologyName'].iloc[i] in bin_one:
        Train['TechnologyComplexity'].iloc[i] = 1
    elif Train['TechnologyName'].iloc[i] in bin_two:
        Train['TechnologyComplexity'].iloc[i] = 2
    elif Train['TechnologyName'].iloc[i] in bin_three:
        Train['TechnologyComplexity'].iloc[i] = 3
    elif Train['TechnologyName'].iloc[i] in bin_four:
        Train['TechnologyComplexity'].iloc[i] = 4
    elif Train['TechnologyName'].iloc[i] in bin_five:
        Train['TechnologyComplexity'].iloc[i] = 5
    else:
        Train['TechnologyComplexity'].iloc[i] = 0
    

# Customer Bins

Here we bin customers into 5 hierarchical bins based on average time to solution for the customer. Bin 1 is low time to solution, Bin 5 is high. The values bin1, bin2, bin3, bin4 will show the boundaries of the bins. 

In [77]:
df = Train_data.groupby(['CustomerNameEncrypted']).mean('TimeToSolution')
bin1 = df['TimeToSolution'].quantile([.20])
bin2 = df['TimeToSolution'].quantile([.40])
bin3 = df['TimeToSolution'].quantile([.60])
bin4 = df['TimeToSolution'].quantile([.80])

c_bin_one = df[(df['TimeToSolution'] > -0.1) & (df['TimeToSolution'] <= bin1.iloc[0])].index
c_bin_two = df[(df['TimeToSolution'] > bin1.iloc[0]) & (df['TimeToSolution'] <= bin2.iloc[0])].index
c_bin_three = df[(df['TimeToSolution'] > bin2.iloc[0]) & (df['TimeToSolution'] <= bin3.iloc[0])].index
c_bin_four = df[(df['TimeToSolution'] > bin3.iloc[0]) & (df['TimeToSolution'] <= bin4.iloc[0])].index
c_bin_five = df[(df['TimeToSolution'] > bin4.iloc[0]) & (df['TimeToSolution'] <= 1000000)].index
Train['CustomerComplexity'] = 0

for i in range (0,len(Train)):
    if Train['CustomerNameEncrypted'].iloc[i] in c_bin_one:
        Train['CustomerComplexity'].iloc[i] = 1
    elif Train['CustomerNameEncrypted'].iloc[i] in c_bin_two:
        Train['CustomerComplexity'].iloc[i] = 2
    elif Train['CustomerNameEncrypted'].iloc[i] in c_bin_three:
        Train['CustomerComplexity'].iloc[i] = 3
    elif Train['CustomerNameEncrypted'].iloc[i] in c_bin_four:
        Train['CustomerComplexity'].iloc[i] = 4
    elif Train['CustomerNameEncrypted'].iloc[i] in c_bin_five:
        Train['CustomerComplexity'].iloc[i] = 5
    else:
        Train['CustomerComplexity'].iloc[i] = 0

# Customer Deal Count

Here we bin customers into 5 hierarchical bins based on the historical deal count for the customer. Bin 1 is low deal count, Bin 5 is high. The values bin1, bin2, bin3, bin4 will show the boundaries of the bins. 

In [78]:
df = Train_data.groupby(['CustomerNameEncrypted']).count()
bin1 = df['TimeToSolution'].quantile([.70])
bin2 = df['TimeToSolution'].quantile([.87])
bin3 = df['TimeToSolution'].quantile([.933])
bin4 = df['TimeToSolution'].quantile([.966])

cd_bin_one = df[(df['TimeToSolution'] > -0.1) & (df['TimeToSolution'] <= bin1.iloc[0])].index
cd_bin_two = df[(df['TimeToSolution'] > bin1.iloc[0]) & (df['TimeToSolution'] <= bin2.iloc[0])].index
cd_bin_three = df[(df['TimeToSolution'] > bin2.iloc[0]) & (df['TimeToSolution'] <= bin3.iloc[0])].index
cd_bin_four = df[(df['TimeToSolution'] > bin3.iloc[0]) & (df['TimeToSolution'] <= bin4.iloc[0])].index
cd_bin_five = df[(df['TimeToSolution'] > bin4.iloc[0]) & (df['TimeToSolution'] <= 1000000)].index
Train['CustomerDealCount'] = 0

for i in range (0,len(Train)):
    if Train['CustomerNameEncrypted'].iloc[i] in cd_bin_one:
        Train['CustomerDealCount'].iloc[i] = 1
    elif Train['CustomerNameEncrypted'].iloc[i] in cd_bin_two:
        Train['CustomerDealCount'].iloc[i] = 2
    elif Train['CustomerNameEncrypted'].iloc[i] in cd_bin_three:
        Train['CustomerDealCount'].iloc[i] = 3
    elif Train['CustomerNameEncrypted'].iloc[i] in cd_bin_four:
        Train['CustomerDealCount'].iloc[i] = 4
    elif Train['CustomerNameEncrypted'].iloc[i] in cd_bin_five:
        Train['CustomerDealCount'].iloc[i] = 5
    else:
        Train['CustomerDealCount'].iloc[i] = 0

# Country Bin


Here we bin countries into 5 hierarchical bins based on the deal count for each country. Bin 1 is low deal count, Bin 5 is high deal count. The values bin1, bin2, bin3, bin4 will show the boundaries of the bins. 

In [79]:
df = Train_data.groupby(['CountryValue']).count()
bin1 = df['TimeToSolution'].quantile([.20])
bin2 = df['TimeToSolution'].quantile([.40])
bin3 = df['TimeToSolution'].quantile([.60])
bin4 = df['TimeToSolution'].quantile([.80])

country_bin_one = df[(df['TimeToSolution'] > -0.1) & (df['TimeToSolution'] <= bin1.iloc[0])].index
country_bin_two = df[(df['TimeToSolution'] > bin1.iloc[0]) & (df['TimeToSolution'] <= bin2.iloc[0])].index
country_bin_three = df[(df['TimeToSolution'] > bin2.iloc[0]) & (df['TimeToSolution'] <= bin3.iloc[0])].index
country_bin_four = df[(df['TimeToSolution'] > bin3.iloc[0]) & (df['TimeToSolution'] <= bin4.iloc[0])].index
country_bin_five = df[(df['TimeToSolution'] > bin4.iloc[0]) & (df['TimeToSolution'] <= 1000000)].index
Train['CountryCount'] = 0

for i in range (0,len(Train)):
    if Train['CountryValue'].iloc[i] in country_bin_one:
        Train['CountryCount'].iloc[i] = 1
    elif Train['CountryValue'].iloc[i] in country_bin_two:
        Train['CountryCount'].iloc[i] = 2
    elif Train['CountryValue'].iloc[i] in country_bin_three:
        Train['CountryCount'].iloc[i] = 3
    elif Train['CountryValue'].iloc[i] in country_bin_four:
        Train['CountryCount'].iloc[i] = 4
    elif Train['CountryValue'].iloc[i] in country_bin_five:
        Train['CountryCount'].iloc[i] = 5
    else:
        Train['CountryCount'].iloc[i] = 0

# OpportunityTypeValue Bin

Here we bin opportunity types into 5 hierarchical bins based on average time to solution for the opportunity type. Bin 1 is low time to solution, Bin 5 is high. The values bin1, bin2, bin3, bin4 will show the boundaries of the bins. 

In [80]:
df = Train_data.groupby(['OpportunityTypeValue']).mean('TimeToSolution')
bin1 = df['TimeToSolution'].quantile([.20])
bin2 = df['TimeToSolution'].quantile([.40])
bin3 = df['TimeToSolution'].quantile([.60])
bin4 = df['TimeToSolution'].quantile([.80])

o_bin_one = df[(df['TimeToSolution'] > -0.1) & (df['TimeToSolution'] <= bin1.iloc[0])].index
o_bin_two = df[(df['TimeToSolution'] > bin1.iloc[0]) & (df['TimeToSolution'] <= bin2.iloc[0])].index
o_bin_three = df[(df['TimeToSolution'] > bin2.iloc[0]) & (df['TimeToSolution'] <= bin3.iloc[0])].index
o_bin_four = df[(df['TimeToSolution'] > bin3.iloc[0]) & (df['TimeToSolution'] <= bin4.iloc[0])].index
o_bin_five = df[(df['TimeToSolution'] > bin4.iloc[0]) & (df['TimeToSolution'] <= 1000000)].index
Train['OpportunityTypeComplexity'] = 0

for i in range (0,len(Train)):
    if Train['OpportunityTypeValue'].iloc[i] in o_bin_one:
        Train['OpportunityTypeComplexity'].iloc[i] = 1
    elif Train['OpportunityTypeValue'].iloc[i] in o_bin_two:
        Train['OpportunityTypeComplexity'].iloc[i] = 2
    elif Train['OpportunityTypeValue'].iloc[i] in o_bin_three:
        Train['OpportunityTypeComplexity'].iloc[i] = 3
    elif Train['OpportunityTypeValue'].iloc[i] in o_bin_four:
        Train['OpportunityTypeComplexity'].iloc[i] = 4
    elif Train['OpportunityTypeValue'].iloc[i] in o_bin_five:
        Train['OpportunityTypeComplexity'].iloc[i] = 5
    else:
        Train['OpportunityTypeComplexity'].iloc[i] = 0

# Subset data based on the variables we want

In [81]:
MyDealData = Train[['Won','RegionValue','Fiscal_Quarter','ContractCosts_Binned','ContractMargin','DeliverableTypeValue',
                   'OpportunityTypeComplexity','Governance','IsFederal','SourceSystem','DealType','HasInnerTechnologies','NumberOfSowRevisions',
                    'DealTechnologies','CountryCount','TechnologyComplexity','CustomerComplexity','CustomerDealCount']]


# data export

It is best to export the data at this point and work from a csv. Otherwise data types may not align in the code below. 

Make sure to replace the path with your own file path. 

In [244]:
### WARNING
MyDealData.to_csv(r"C:\Users\justi\OneDrive\Documents\MyDeal_transformed.csv")
### WARNING

# Import Clean data


Make sure to replace the path with your own file path. 

In [245]:
import pandas as pd
MyDealData = pd.read_csv(r"C:\Users\justi\OneDrive\Documents\MyDeal_transformed.csv") 
MyDealData = MyDealData.drop('Unnamed: 0',1)



# Grab info on categoricals so we can transform new data later on

For the categoricals that we will one-hot encode, we need to save the index of levels it takes on in the training set. 

In [121]:
regions = MyDealData.RegionValue.value_counts().index.tolist()
source = MyDealData.SourceSystem.value_counts().index.tolist()
dealtype = MyDealData.DealType.value_counts().index.tolist()
deliverabletype = MyDealData.DeliverableTypeValue.value_counts().index.tolist()

# Getting Dummies

One Hot encoding for the categorical variables that remain.

In [122]:
#Note that we aren't taking dummies for the binned variables, they are treated as ordinal for the tree model
MyDealData = pd.get_dummies(data=MyDealData, columns = ['RegionValue','SourceSystem','DealType','DeliverableTypeValue'
                    ], \
                                   prefix = ['RegionValue','SourceSystem','DealType','DeliverableTypeValue'
                    ])

Here we split the data into X and y matrices.

In [126]:
Train_data = MyDealData

X_train = Train_data.drop('Won',1)
y_train = Train_data['Won'].values

# Random Forest Cross Validation

### Only Need to run this feature tuning section once, then fit model with the best params 

The following two chunks runs cross validation over a grid of different parameters to choose the Random Forest Parameters that optimizer performance. 

In [24]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [25]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:   30.0s remaining:   26.3s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

Here we can look at the parameters chosen through gridsearchCV. 

In [47]:
rf_random.best_params_

{'n_estimators': 452,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 89,
 'bootstrap': False}

# Random Forest Model

Now we can train our random forest model on the training data with the best parameters chosen from above (note that they are hardcoded). We check accuracy on the training set. 

In [252]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 452, min_samples_split = 10, min_samples_leaf = 4, max_depth=89,max_features= 'sqrt', random_state=0)
clf.fit(X_train, y_train)

prediction_train = clf.predict(X_train)
print('Classification Report: Train')
print(classification_report(y_train, prediction_train))

Classification Report: Train
              precision    recall  f1-score   support

           0       0.80      0.71      0.75     11440
           1       0.77      0.85      0.81     13137

    accuracy                           0.78     24577
   macro avg       0.79      0.78      0.78     24577
weighted avg       0.79      0.78      0.78     24577



# Functions to transform new data

This function is designed to take in data from an excel file and transform it into the format that the model takes in: specifying contract cost bin, one-hot encoding categoricals, etc. 

In [63]:
def transform(x):
    x['ContractCosts_Binned'] = 0
    x['TechnologyComplexity'] = 0
    x['CustomerComplexity'] = 0
    x['CustomerDealCount'] = 0
    x['CountryCount'] = 0
    x['OpportunityTypeComplexity'] = 0
    x['ContractCost'].fillna(mean_cc)
    x['ContractMargin'].fillna(mean_cm)
    for i in range (0,(len(x))):
        if x.ContractCost[i] <= 4181.75:
            x.ContractCosts_Binned[i] = 1
        elif x.ContractCost[i] <= 12357:
            x.ContractCosts_Binned[i] = 2
        elif x.ContractCost[i] <= 38167:
            x.ContractCosts_Binned[i] = 3
        else:
            x.ContractCosts_Binned[i] = 4
        try:
            int(x.NumberOfSowRevisions[i])
        except:
            x.NumberOfSowRevisions[i] = 0
#         if x.ContractMargin[i] <= 0.29:
#             x.ContractMargin_Binned[i] = 1
#         elif x.ContractMargin[i] <= 0.4:
#             x.ContractMargin_Binned[i] = 2
#         elif x.ContractMargin[i] <= 0.5:
#             x.ContractMargin_Binned[i] = 3
#         else:
#             x.ContractMargin_Binned[i] = 4
        if x['TechnologyName'][i] in bin_one:
            x['TechnologyComplexity'][i] = 1
        elif x['TechnologyName'][i] in bin_two:
            x['TechnologyComplexity'][i] = 2
        elif x['TechnologyName'][i] in bin_three:
            x['TechnologyComplexity'][i] = 3
        elif x['TechnologyName'][i] in bin_four:
            x['TechnologyComplexity'][i] = 4
        elif x['TechnologyName'][i] in bin_five:
            x['TechnologyComplexity'][i] = 5
        else:
            x['TechnologyComplexity'][i] = 0
        if x['CustomerNameEncrypted'][i] in c_bin_one:
            x['CustomerComplexity'][i] = 1
        elif x['CustomerNameEncrypted'][i] in c_bin_two:
            x['CustomerComplexity'][i] = 2
        elif x['CustomerNameEncrypted'][i] in c_bin_three:
            x['CustomerComplexity'][i] = 3
        elif x['CustomerNameEncrypted'][i] in c_bin_four:
            x['CustomerComplexity'][i] = 4
        elif x['CustomerNameEncrypted'][i] in c_bin_five:
            x['CustomerComplexity'][i] = 5
        else:
            x['CustomerComplexity'][i] = 0
        if x['CustomerNameEncrypted'][i] in cd_bin_one:
            x['CustomerDealCount'][i] = 1
        elif x['CustomerNameEncrypted'][i] in cd_bin_two:
            x['CustomerDealCount'][i] = 2
        elif x['CustomerNameEncrypted'][i] in cd_bin_three:
            x['CustomerDealCount'][i] = 3
        elif x['CustomerNameEncrypted'][i] in cd_bin_four:
            x['CustomerDealCount'][i] = 4
        elif x['CustomerNameEncrypted'][i] in cd_bin_five:
            x['CustomerDealCount'][i] = 5
        else:
            x['CustomerDealCount'][i] = 0
        if x['CountryValue'][i] in country_bin_one:
            x['CountryCount'][i] = 1
        elif x['CountryValue'][i] in country_bin_two:
            x['CountryCount'][i] = 2
        elif x['CountryValue'][i] in country_bin_three:
            x['CountryCount'][i] = 3
        elif x['CountryValue'][i] in country_bin_four:
            x['CountryCount'][i] = 4
        elif x['CountryValue'][i] in country_bin_five:
            x['CountryCount'][i] = 5
        else:
            x['CountryCount'][i] = 0
        if x['OpportunityTypeValue'][i] in o_bin_one:
            x['OpportunityTypeComplexity'][i] = 1
        elif x['OpportunityTypeValue'][i] in o_bin_two:
            x['OpportunityTypeComplexity'][i] = 2
        elif x['OpportunityTypeValue'][i] in o_bin_three:
            x['OpportunityTypeComplexity'][i] = 3
        elif x['OpportunityTypeValue'][i] in o_bin_four:
            x['OpportunityTypeComplexity'][i] = 4
        elif x['OpportunityTypeValue'][i] in o_bin_five:
            x['OpportunityTypeComplexity'][i] = 5
        else:
            x['OpportunityTypeComplexity'][i] = 0
        def month(x):
            x = x.month
            return x
        x['Fiscal_Quarter'] = x.IntakeDate.map(month)
        x['Fiscal_Quarter'] = x.Fiscal_Quarter.replace([2,3,4,5,6,7,8,9,10,11,12,1],[1,1,1,2,2,2,3,3,3,4,4,4])
        x['Governance'] = np.where(x['GovernanceStatusName']== 'No Governance', 0, 1)
        
        complexity = x.groupby('MyDealId',as_index=True).size()
        complexity_dict = complexity.to_dict()
        
        x['DealTechnologies'] = x['MyDealId'].map(complexity_dict)
        
        
        x['SourceSystem'] = x['SourceSystem'].fillna('SFDC - Dell Main')
        x['NumberOfSowRevisions'] = x.NumberOfSowRevisions.fillna(0)
    return x[['RegionValue','Fiscal_Quarter','ContractCosts_Binned','ContractMargin','DeliverableTypeValue',
                   'OpportunityTypeComplexity','Governance','IsFederal','SourceSystem','DealType','HasInnerTechnologies','NumberOfSowRevisions',
                    'DealTechnologies','CountryCount','TechnologyComplexity','CustomerComplexity','CustomerDealCount','IsPrimary']]
        
def transform2(x):
    for i in regions:
        x['RegionValue_'+i] = 0
    for i in regions:
        for j in range(0,len(x)):
            if x['RegionValue'][j] ==i:
                x['RegionValue_'+i][j] = 1
            else:
                x['RegionValue_'+i][j] = 0
    for i in source:
        x['SourceSystem_'+i] = 0
    for i in source:
        for j in range(0,len(x)):
            if x['SourceSystem'][j] ==i:
                x['SourceSystem_'+i][j] = 1
            else:
                x['SourceSystem_'+i][j] = 0
    for i in dealtype:
        x['DealType_'+i] = 0
    for i in dealtype:
        for j in range(0,len(x)):
            if x['DealType'][j] ==i:
                x['DealType_'+i][j] = 1
            else:
                x['DealType_'+i][j] = 0
    for i in deliverabletype:
        x['DeliverableTypeValue_'+i] = 0
    for i in deliverabletype:
        for j in range(0,len(x)):
            if x['DeliverableTypeValue'][j] ==i:
                x['DeliverableTypeValue_'+i][j] = 1
            else:
                x['DeliverableTypeValue_'+i][j] = 0
    df_new = x.drop(['RegionValue','SourceSystem','DealType','DeliverableTypeValue'],1)
    return df_new

    

# Read in What-if Scenario

Import the set of deals you want win probability predictions for. Replace the file path with your own. 

In [247]:
new_deals = pd.read_excel(r"C:\Users\justi\OneDrive\Documents\final_demo1.xlsx") 

# Transform the new data

This first cell runs the functions above on the input data to create the variables needed for the model. 

In [248]:
new1 = transform(new_deals)
new2 = transform2(new1)

This cell reorders the columns of the new data to match the order used in the original training of the model. 

In [249]:
#unscaled data sequence
X_test = new2
prim = new2.IsPrimary

#this line reorders columns
X_test = new2.drop(['IsPrimary'],1)[X_train.columns]

#re-add IsPrimary
X_test['IsPrimary'] = prim

# Predict Probability deal will be won

Finally, we output the win probability for the hypothetical deal. 

In [251]:
for i in range(len(X_test)):
    if X_test.IsPrimary[i] == 1:
        print(clf.predict_proba(X_test.drop(['IsPrimary'],1))[i][1])
    else:
        print(clf.predict_proba(X_test.drop(['IsPrimary'],1))[i][1],'*warning* part of a parent deal')

Win Probability
0.5420736457073909
